In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys
sys.path.append('../..')

In [112]:
import asyncio
import logging

from app.redis_transcribe.connection import get_redis_client
from app.settings import settings
from app.services.transcription.processor import Processor




redis_client = await get_redis_client(settings.redis_host, settings.redis_port,settings.redis_password)

processor = Processor(redis_client,max_length=30)



2025-02-14 17:19:00,726 - INFO - Attempting Redis connection to redis:6379
2025-02-14 17:19:00,727 - INFO - Redis password is configured
2025-02-14 17:19:00,729 - INFO - Successfully connected to Redis server


In [113]:
from app.services.audio.redis_models import Transcript

In [139]:
df = await Transcript.get_all_transcripts_df(redis_client)
df.tail(10)

,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,words,meeting_id
6,"Нет, точнее он будет open-source полностью, ч...",2025-02-13 15:44:24.424000+00:00,2025-02-13 15:44:40.364000+00:00,Dmitriy Grankin,0.690088,1,"[[0.0, 0.48, Нет,, 0.7353515625], [0.54, 0.88...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
7,"Подожди, подожди, подожди. Единственное, что....",2025-02-13 15:44:40.984000+00:00,2025-02-13 15:44:54.704000+00:00,Sergey Ryabenko,0.707289,2,"[[16.56, 17.04, Подожди,, 0.8976236979166666]...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
8,Ты не можешь вырезать оттуда рекламу. И второ...,2025-02-13 15:44:54.844000+00:00,2025-02-13 15:45:10.244000+00:00,Sergey Ryabenko,0.659481,3,"[[30.42, 30.54, Ты, 0.9775390625], [30.54, 30...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
9,"Ну, там, да, такая лицензия на...",2025-02-13 15:45:10.364000+00:00,2025-02-13 15:45:12.904000+00:00,Sergey Ryabenko,0.250394,4,"[[45.94, 46.64, Ну,, 0.794921875], [46.68, 47...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
2,"Наверное, будет, что если ты сам для себя раз...",2025-02-13 15:45:13.180000+00:00,2025-02-13 15:45:22.020000+00:00,Dmitriy Grankin,1.000000,1,"[[0.18, 0.26, Наверное,, 0.6663665771484375],...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
3,"Ты разворачивай, пожалуйста, пользуйся. Если ...",2025-02-13 15:45:22.320000+00:00,2025-02-13 15:45:39.380000+00:00,Dmitriy Grankin,0.608441,2,"[[9.32, 9.88, Ты, 0.52294921875], [9.88, 11.2...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
4,"Ага. Ну, просто чтобы... Ну, да, да, не получ...",2025-02-13 15:45:39.380000+00:00,2025-02-13 15:45:52.020000+00:00,Dmitriy Grankin,0.365506,3,"[[26.38, 27.6, Ага., 0.40765380859375], [27.6...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
5,"Ага. Ну, да, логично, логично. Вот, ну, слуша...",2025-02-13 15:45:52.760000+00:00,2025-02-13 15:46:12.500000+00:00,Sergey Ryabenko,0.405268,4,"[[39.76, 40.14, Ага., 0.9638671875], [42.75, ...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
0,На парковку может звук.,2025-02-13 15:46:13.164000+00:00,2025-02-13 15:46:14.644000+00:00,Dmitriy Grankin,0.564865,1,"[[0.5, 0.94, На, 0.443359375], [0.94, 1.42, ...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700
1,"Давай, пока.",2025-02-13 15:46:15.224000+00:00,2025-02-13 15:46:16.044000+00:00,Sergey Ryabenko,0.053659,2,"[[2.56, 3.04, Давай,, 0.99169921875], [3.14, ...",mock_user_2490f037-c1f5-4051-96c9-b9217bcf4700


In [ ]:
#52:30